In [18]:
from dcicutils import ff_utils
from functions.notebook_functions import *
from functions.cleanup import *
import time
time1 = time.time()

my_auth = get_key('koray_data')

change_status = 'released to project'
# 4DNESXE9K9DB
# 4DNESUJZ5FL2
all_sets = [ff_utils.get_metadata('4DNESUJZ5FL2', my_auth)]
# ready_sets_1 = [i for i in all_sets if "HiC_Pipeline_0.2.5" in i.get('completed_processes', [])]
set_to_release = [i['uuid'] for i in all_sets]

store={}
item_uuids=[]
store, uuids = ff_utils.expand_es_metadata(set_to_release, my_auth, store_frame='object',add_pc_wfr=True, ignore_field = ['experiment_relation', 'biosample_relation', 'references'])
print(len(store['experiment_set_replicate']), 'exp sets for status change')
print(len(uuids), 'items collected')
time2 = time.time()
print(round((time2-time1), 1), 'sec for collection')

common_types = ['lab', 'user', 'award','ontology', 'ontology_term', 
                'file_format', 'software', 'workflow', 'static_section',
                'organism', 'vendor', 'file_reference', 'individual_human', 
                'enzyme', 'biosource', 'file_format']

skip_status = ['replaced', 'restricted', 'archived', 'revoked', 'deleted']

1 exp sets for status change
80 items collected
2.5 sec for collection


In [19]:
# TODO
# Check audits

# do you want to check for duplicate/problematic runs on files?
# it will take some time
check_wfrs = True
delete_problematic = False

# check expsets
print('EXPSET CHECK')
for a_set in store['experiment_set_replicate']:
    if not a_set.get('completed_processes'):
        print(a_set['accession'], 'missing processing tag', a_set['description'][:50])

# check exps 
print('\nEXP CHECK')
# check for experiment numbers
exp_names = [i for i in store if i.startswith('experiment') and not i.startswith('experiment_set')]
all_exps_on_sets = [a for i in store['experiment_set_replicate'] for a in i['experiments_in_set']]
all_exps = [a['uuid'] for i in store.keys() for a in store[i] if i in exp_names]
if len(all_exps_on_sets) != len(all_exps):
    print('Number of experiments is not same as experiments associated with sets')
    print('# of exps: {}. # of exps on sets: {}'.format(len(all_exps), len(all_exps_on_sets)))
        
print('\nFILE FASTQ CHECK')
for a_file in store['file_fastq']:
    if not a_file.get('quality_metric'):
        print(a_file['accession'], 'missing fastqc')
    if not a_file.get('content_md5sum') or not a_file.get('md5sum'):
        print(a_file['accession'], 'problems with md5')
    if check_wfrs:
        dw = delete_wfrs(a_file, my_auth, delete=delete_problematic)

# check processed files
print('\nFILE PROCESSED CHECK')
if store.get('file_processed'):
    for a_file in store['file_processed']:
        if a_file['file_format'] == '/file-formats/pairs/':
            if not a_file.get('quality_metric'):
                print(a_file['accession'], 'missing Pairsqc')
        if not a_file.get('source_experiments'):
            print(a_file['accession'], 'missing source experiments')
        if check_wfrs:
            dw = delete_wfrs(a_file, my_auth, delete=delete_problematic)   

# check wfrs
print('\nWFR CHECK')
if store.get('workflow_run_awsem'):
    for wfr in store['workflow_run_awsem']:
        if wfr['run_status'] != 'complete':
            print('problematic wfr', wfr['uuid'], wfr['run_status'])
        
# check for weird status
print('\nREPORT NUMBERS AND CHECK STATUS')
for i in store:
    print(i, len(store[i]))
    weird = [[i,x['uuid'],x['status']] for x in store[i] if x['status'] in skip_status]
    if weird:
        for case in weird:
            print(case)
        print()


EXPSET CHECK
4DNESUJZ5FL2 missing processing tag WGS seq of total genomic DNA from crosslinked F121

EXP CHECK

FILE FASTQ CHECK

FILE PROCESSED CHECK

WFR CHECK

REPORT NUMBERS AND CHECK STATUS
organism 1
individual_mouse 1
experiment_set_replicate 1
award 3
user 7
experiment_seq 2
lab 3
software 1
biosource 1
ontology 2
vendor 1
workflow_run_awsem 16
ontology_term 10
file_fastq 8
workflow 3
protocol 4
image 2
biosample_cell_culture 2
file_format 2
quality_metric_fastqc 8
biosample 2


In [20]:
# Check status
print_each = False
print('COMMON ITEMS')
for a_type in store:
    if a_type in common_types:
        for raw_data in store[a_type]:
            if raw_data['status'] not in ['released', 'current']:
                print('COMMON ITEM NOT RELEASED', a_type,raw_data['status'], raw_data['uuid'])
print('---------------------------------\n')
counter = 0
for a_type in store:
    total = len(store[a_type])
    change = 0
    released = 0
    matching = 0
    unusual = 0
    if a_type in common_types:
        continue
    for raw_data in store[a_type]:
        if raw_data['status'] in ['released', 'current']:
            released += 1
            msg = ('ALREADY RELEASED ITEM', a_type, raw_data['uuid'], raw_data['status'])
        elif raw_data['status'] == change_status:
            matching += 1
            msg = ('ALREADY TARGET STATUS', a_type, raw_data['uuid'], raw_data['status'])
        elif raw_data['status'] in skip_status:
            unusual += 1
            msg = ('SKIP UNUSUAL STATUS  ', a_type, raw_data['uuid'], raw_data['status'])
        else:
            change += 1
            msg = ('       CHANGE        ', a_type, raw_data['uuid'], raw_data['status'])
        if print_each:
            print(' '.join(msg))
    print('{:<25} Out of {t}, {r} released, {m} matching, {u} unusual, and {c}needs change'.format(a_type, t=total, r=released, m=matching, u=unusual, c=change))
    
        


COMMON ITEMS
---------------------------------

individual_mouse          Out of 1, 1 released, 0 matching, 0 unusual, and 0needs change
experiment_set_replicate  Out of 1, 0 released, 0 matching, 0 unusual, and 1needs change
experiment_seq            Out of 2, 0 released, 0 matching, 0 unusual, and 2needs change
workflow_run_awsem        Out of 16, 0 released, 0 matching, 0 unusual, and 16needs change
file_fastq                Out of 8, 0 released, 0 matching, 0 unusual, and 8needs change
protocol                  Out of 4, 2 released, 2 matching, 0 unusual, and 0needs change
image                     Out of 2, 0 released, 2 matching, 0 unusual, and 0needs change
biosample_cell_culture    Out of 2, 0 released, 2 matching, 0 unusual, and 0needs change
quality_metric_fastqc     Out of 8, 0 released, 0 matching, 0 unusual, and 8needs change
biosample                 Out of 2, 0 released, 2 matching, 0 unusual, and 0needs change


In [21]:
# Check status

action = True


# Check status
print_each = False
print('COMMON ITEMS')
for a_type in store:
    if a_type in common_types:
        for raw_data in store[a_type]:
            if raw_data['status'] not in ['released', 'current']:
                print('COMMON ITEM NOT RELEASED', a_type,raw_data['status'], raw_data['uuid'])
print('---------------------------------\n')
counter = 0
for a_type in store:
    total = len(store[a_type])
    change = 0
    released = 0
    matching = 0
    unusual = 0
    if a_type in common_types:
        continue
    for raw_data in store[a_type]:
        if raw_data['status'] in ['released', 'current']:
            released += 1
            msg = ('ALREADY RELEASED ITEM', a_type, raw_data['uuid'], raw_data['status'])
        elif raw_data['status'] == change_status:
            matching += 1
            msg = ('ALREADY TARGET STATUS', a_type, raw_data['uuid'], raw_data['status'])
        elif raw_data['status'] in skip_status:
            unusual += 1
            msg = ('SKIP UNUSUAL STATUS  ', a_type, raw_data['uuid'], raw_data['status'])
        else:
            change += 1
            msg = ('       CHANGE        ', a_type, raw_data['uuid'], raw_data['status'])
            if action:
                patch_data = {'status': change_status}
                if change_status == 'released':
                    if a_type in ['publication']:
                        ff_utils.patch_metadata({'status': 'current'}, obj_id=raw_data['uuid'] ,key=my_auth)
                    else:
                        ff_utils.patch_metadata(patch_data, obj_id=raw_data['uuid'] ,key=my_auth)
                else:
                    ff_utils.patch_metadata(patch_data, obj_id=raw_data['uuid'] ,key=my_auth)    
        if print_each:
            print(' '.join(msg))
    print('{:<25} Out of {t}, {r} released, {m} matching, {u} unusual, and {c} UPDATED with status'.format(a_type, t=total, r=released, m=matching, u=unusual, c=change))

COMMON ITEMS
---------------------------------

individual_mouse          Out of 1, 1 released, 0 matching, 0 unusual, and 0 UPDATED with status
experiment_set_replicate  Out of 1, 0 released, 0 matching, 0 unusual, and 1 UPDATED with status
experiment_seq            Out of 2, 0 released, 0 matching, 0 unusual, and 2 UPDATED with status
workflow_run_awsem        Out of 16, 0 released, 0 matching, 0 unusual, and 16 UPDATED with status
file_fastq                Out of 8, 0 released, 0 matching, 0 unusual, and 8 UPDATED with status
protocol                  Out of 4, 2 released, 2 matching, 0 unusual, and 0 UPDATED with status
image                     Out of 2, 0 released, 2 matching, 0 unusual, and 0 UPDATED with status
biosample_cell_culture    Out of 2, 0 released, 2 matching, 0 unusual, and 0 UPDATED with status
quality_metric_fastqc     Out of 8, 0 released, 0 matching, 0 unusual, and 8 UPDATED with status
biosample                 Out of 2, 0 released, 2 matching, 0 unusual, and 0 